In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import matplotlib.pyplot as plt
import numpy as np

In [2]:
(train_images, _), (_, _) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 784).astype('float32')
train_images = (train_images - 127.5) / 127.5  

11490434/11490434 [==============================] - 2s 0us/step


In [3]:
def build_generator():
    model = models.Sequential([
        layers.Dense(256, input_dim=100, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(1024, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(784, activation='tanh')
    ])
    return model

In [4]:
def build_discriminator():
    model = models.Sequential([
        layers.Dense(1024, input_dim=784, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

In [5]:
def build_gan(generator, discriminator):
    model = models.Sequential([
        generator,
        discriminator
    ])
    return model

In [6]:
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(0.0002, 0.5), metrics=['accuracy'])

# Construir el Generador
generator = build_generator()

# El Discriminador no se debe entrenar durante el entrenamiento del Generador
discriminator.trainable = False

# Construir y compilar la GAN
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(0.0002, 0.5))

# Entrenar la GAN
epochs = 20000
batch_size = 64
half_batch = batch_size // 2

for epoch in range(epochs):
    
    # Entrenar el Discriminador
    idx = np.random.randint(0, train_images.shape[0], half_batch)
    imgs = train_images[idx]
    
    noise = np.random.normal(0, 1, (half_batch, 100))
    gen_imgs = generator.predict(noise)
    
    d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
    d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    
    # Entrenar el Generador
    noise = np.random.normal(0, 1, (batch_size, 100))
    valid_y = np.array([1] * batch_size)
    g_loss = gan.train_on_batch(noise, valid_y)
    
    # Mostrar el progreso
    if epoch % 1000 == 0:
        print(f"{epoch} [D loss: {d_loss[0]} | D accuracy: {100*d_loss[1]}] [G loss: {g_loss}]")


1/1 [==============================] - 0s 261ms/step
0 [D loss: 0.8035248816013336 | D accuracy: 37.5] [G loss: 0.555721640586853]
1/1 [==============================] - 0s 20ms/step
1000 [D loss: 0.495557501912117 | D accuracy: 73.4375] [G loss: 1.634270191192627]
1/1 [==============================] - 0s 26ms/step
2000 [D loss: 0.6345044672489166 | D accuracy: 67.1875] [G loss: 1.191334843635559]
1/1 [==============================] - 0s 32ms/step
3000 [D loss: 0.6030395030975342 | D accuracy: 65.625] [G loss: 1.144521713256836]
1/1 [==============================] - 0s 30ms/step
4000 [D loss: 0.6265127658843994 | D accuracy: 59.375] [G loss: 0.9745845794677734]
1/1 [==============================] - 0s 29ms/step
5000 [D loss: 0.6042075455188751 | D accuracy: 60.9375] [G loss: 0.852243959903717]
1/1 [==============================] - 0s 32ms/step
6000 [D loss: 0.6399248838424683 | D accuracy: 60.9375] [G loss: 0.7809544205665588]
1/1 [==============================] - 0s 35ms/step
70

KeyboardInterrupt: 